# Change Log
---
Tested CNN and and RNNwithAttention
aDDED MERGE LAYER
aDDED FFN AR THE END

In [15]:
import sys
sys.path.append('/home/sunil.patel/PycharmProjects/extensibleFramework/')

In [50]:
from extensibleFramework.src.models import convolution_neural_network
from extensibleFramework.src.models import recurrent_nn_with_attention
from extensibleFramework.src.models import feed_forward_network
from extensibleFramework.src.models import extra_layers

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
from torchtext import data
import nltk
import json
from torchtext import vocab
from tqdm import tqdm
import torch
import random
import re
import os
import numpy as np
import sys
import random
import tarfile
import urllib
from torchtext import data
import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [52]:
# # DataSource : http://www.cs.cornell.edu/people/pabo/movie-review-data/
# !wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
# !tar -xvf rt-polaritydata.tar.gz

In [53]:
# SEED = 1
# split = 0.80
# data_block = []
# negative_data  = open('rt-polaritydata/rt-polarity.neg',encoding='utf8',errors='ignore').read().splitlines()
# for i in negative_data:
#         data_block.append({"sentiment":str(i.strip()),"label" : 0}) 
# positve_data  = open('rt-polaritydata/rt-polarity.pos',encoding='utf8',errors='ignore').read().splitlines()
# for i in positve_data:
#         data_block.append({"sentiment":str(i.strip()),"label" : 1}) 

In [54]:
# random.shuffle(data_block)

# train_file = open('train.json', 'w')
# test_file = open('test.json', 'w')
# for i in  range(0,int(len(data_block)*split)):
#     train_file.write(str(json.dumps(data_block[i]))+"\n")
# for i in  range(int(len(data_block)*split),len(data_block)):
#     test_file.write(str(json.dumps(data_block[i]))+"\n")

In [55]:
def tokenize(sentiments):
#     print(sentiments)
    return sentiments
def pad_to_equal(x):
    if len(x) < 61:
        return x + ['<pad>' for i in range(0, 61 - len(x))]
    else:
        return x[:61]
def to_categorical(x):
    if x == 1:
        return [0,1]
    if x == 0:
        return [1,0]

In [56]:
SENTIMENT = data.Field(sequential=True , preprocessing =pad_to_equal , use_vocab = True, lower=True,batch_first=True)
LABEL = data.Field(is_target=True,use_vocab = False, sequential=False, preprocessing =to_categorical)
fields = {'sentiment': ('sentiment', SENTIMENT), 'label': ('label', LABEL)}
train_data , test_data = data.TabularDataset.splits(
                            path = '',
                            train = 'train.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields)

SENTIMENT.build_vocab(train_data, test_data)
LABEL.build_vocab(train_data, test_data)
train_iter, test_iter = data.Iterator.splits(
        (train_data, test_data), sort_key=lambda x: len(x.sentiment),
        batch_sizes=(16,len(test_data)), device=device,)

In [57]:
for batch in train_iter:
    feature, target = batch.sentiment, batch.label
    print(feature.data.shape, target.data.shape)
    break

torch.Size([16, 61]) torch.Size([16, 2])


In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class ensemble_model(nn.Module):
    def __init__(self, vocab_size, embed_dim, class_num, \
                 out_channel_num, kernel_size, dropout, input_size, hidden_size, batch_size, n_layers=1, rnn_unit="GRU"):
        super(ensemble_model, self).__init__() 
        self.cnn = convolution_neural_network.cnn_text(vocab_size, embed_dim, class_num, out_channel_num, kernel_size, dropout)
        self.rnn_attention = recurrent_nn_with_attention.RNNAttentionModel(output_size=class_num, hidden_size=hidden_size,\
                                                                          vocab_size=vocab_size, embedding_length=embed_dim, \
                                                                          batch_size=batch_size)
        self.merge_layer = extra_layers.MergeAndFlattern(mode="CONCAT",batch_size=batch_size)
        self.ffn = feed_forward_network.ffn(activation = "Relu", num_layer = 3 , input_size = 2000 , output_size = 2, perceptron_per_layer = [1000,500,250] , dropout = [0.2,0.2,0.2])
    
    def forward(self, x):
        cnn_output = self.cnn(x)
        rnnAttention_output = self.rnn_attention(x)
        merge_layer_output = self.merge_layer(cnn_output,rnnAttention_output )
        final_output = self.ffn(merge_layer_output)
        print("cnn_output : ",cnn_output.shape)
        print("rnn_input : ",rnnAttention_output.shape)
        print("merge_layer_output : ", merge_layer_output.shape)
        print("final_output : ", final_output.shape)

In [68]:
input_size = 10
hidden_size = 128
batch_size = 8
n_layers = 1
vocab_size = 5
embed_dim  = 100
class_num =  1000
out_channel_num = 8
kernel_sizes = [2,3,4]
dropout = 0.5

EM  = ensemble_model(vocab_size=vocab_size, embed_dim=embed_dim, class_num=class_num,out_channel_num=out_channel_num,\
                    kernel_size=kernel_sizes, dropout=dropout, input_size = input_size, hidden_size=hidden_size,\
                    batch_size = batch_size, n_layers = n_layers, rnn_unit = "GRU")


In [69]:
cnn_input = torch.Tensor(np.random.randint(5, size=[8, 10])).long()
rnn_input = torch.Tensor(np.random.random([8, 10])).long() #batch_size, input_size

In [70]:
EM(cnn_input)

cnn_output :  torch.Size([8, 1000])
rnn_input :  torch.Size([8, 1000])
merge_layer_output :  torch.Size([8, 2000])
final_output :  torch.Size([8, 2])
